In [1]:
import os
from pyprojroot import here
os.environ['TRANSFORMERS_CACHE'] = str(here("cache/transformers"))

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "openai-community/gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
model

c:\Users\Plancha\Desktop\AML-homework\.pixi\envs\gpu\Lib\site-packages\transformers\utils\hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [2]:
# test one example
model.eval()
# Add padding token if not already present
tokenizer.add_special_tokens({'pad_token': '<|pad|>'})
model.resize_token_embeddings(len(tokenizer))

# Generate text
prompt = "The future of artificial intelligence is"
inputs = tokenizer(prompt, return_tensors="pt")

# Generate with different parameters
with torch.no_grad():
  outputs = model.generate(
    inputs.input_ids,
    max_new_tokens=100,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
    do_sample=True,
    temperature=0.7
  )

# Decode and display
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


The future of artificial intelligence is still in flux, but at least the main contenders are still in the process of being designed.

What does this mean for you?

It means that people with AI can be more productive and more productive at the same time. There's nothing wrong with that. What's more, it's not necessarily wrong to have an AI that can answer all of the social questions you're facing.

And it might also mean that you don't have to worry about any of the social problems


In [3]:
# dataset from https://recipenlg.cs.put.poznan.pl/
import pandas as pd
import duckdb as sql
data = sql.query(f"""
  select title, directions
  from read_csv(
    '{here('data/recipeNLG/recipeNLG.csv')}', 
    header=True,
    delim=',',
    types = {{
      'title': 'VARCHAR',
      'ingredients': 'VARCHAR[]',
      "directions": 'VARCHAR[]'
    }}
  )
  limit 1000
""")
data = data.df()
data

,title,directions
0,No-Bake Nut Cookies,"[In a heavy 2-quart saucepan, mix brown sugar,..."
1,Jewell Ball'S Chicken,"[Place chipped beef on bottom of baking dish.,..."
2,Creamy Corn,"[In a slow cooker, combine all ingredients. Co..."
3,Chicken Funny,"[Boil and debone chicken., Put bite size piece..."
4,Reeses Cups(Candy),[Combine first four ingredients and press in 1...
...,...,...
995,Heath Bar Pie,[Mix chopped Heath bars with whipped topping a...
996,Victorian Baked French Toast,"[Cook brown sugar, butter and corn syrup in sm..."
997,Quick Swedish Meatballs,"[Combine meat, bread crumbs, cheese, soup mix,..."
998,Irish Stew(Microwave),"[In 4-quart casserole, combine lamb, 1 1/4 cup..."


In [ ]:
from tqdm.notebook import tqdm

tqdm.pandas(desc="Formatting directions")

# in directions, transform list to string, with every line starting with "1 -"
data['directions'] = data['directions'].progress_apply(
  lambda x: '\n'.join([f"{i+1} - {step}" for i, step in enumerate(x)])
)
data['formatted'] = data.progress_apply(
  lambda row: f"Recipe: {row['title']}\n\nInstructions:\n{row['directions']}\n{tokenizer.eos_token}",
  axis=1
)
data

Formatting directions:   0%|          | 0/1000 [00:00<?, ?it/s]

Formatting directions:   0%|          | 0/1000 [00:00<?, ?it/s]

,title,directions,formatted
0,No-Bake Nut Cookies,"1 - In a heavy 2-quart saucepan, mix brown sug...",Recipe: No-Bake Nut Cookies\n\nInstructions:\n...
1,Jewell Ball'S Chicken,1 - Place chipped beef on bottom of baking dis...,Recipe: Jewell Ball'S Chicken\n\nInstructions:...
2,Creamy Corn,"1 - In a slow cooker, combine all ingredients....",Recipe: Creamy Corn\n\nInstructions:\n1 - In a...
3,Chicken Funny,1 - Boil and debone chicken.\n2 - Put bite siz...,Recipe: Chicken Funny\n\nInstructions:\n1 - Bo...
4,Reeses Cups(Candy),1 - Combine first four ingredients and press i...,Recipe: Reeses Cups(Candy) \n\nInstructions:\...
...,...,...,...
995,Heath Bar Pie,1 - Mix chopped Heath bars with whipped toppin...,Recipe: Heath Bar Pie\n\nInstructions:\n1 - Mi...
996,Victorian Baked French Toast,"1 - Cook brown sugar, butter and corn syrup in...",Recipe: Victorian Baked French Toast\n\nInstru...
997,Quick Swedish Meatballs,"1 - Combine meat, bread crumbs, cheese, soup m...",Recipe: Quick Swedish Meatballs\n\nInstruction...
998,Irish Stew(Microwave),"1 - In 4-quart casserole, combine lamb, 1 1/4 ...",Recipe: Irish Stew(Microwave) \n\nInstruction...


In [5]:
print(data["formatted"][0])

Recipe: No-Bake Nut Cookies

Instructions:
1 - In a heavy 2-quart saucepan, mix brown sugar, nuts, evaporated milk and butter or margarine.
2 - Stir over medium heat until mixture bubbles all over top.
3 - Boil and stir 5 minutes more. Take off heat.
4 - Stir in vanilla and cereal; mix well.
5 - Using 2 teaspoons, drop and shape into 30 clusters on wax paper.
6 - Let stand until firm, about 30 minutes.
<|endoftext|>


In [6]:
from torch.utils.data import Dataset
# Custom Dataset class
class RecipeDataset(Dataset):
  def __init__(self, texts, tokenizer, max_length=512):
    self.texts = texts
    self.tokenizer = tokenizer
    self.max_length = max_length
      
  def __len__(self):
    return len(self.texts)
  
  def __getitem__(self, idx):
    text = self.texts[idx]
    encoding = self.tokenizer(
      text,
      truncation=True,
      padding='max_length',
      max_length=self.max_length,
      return_tensors='pt'
    )
    
    return {
      'input_ids': encoding['input_ids'].flatten(),
      'attention_mask': encoding['attention_mask'].flatten()
    }

dataset = RecipeDataset(
  texts=data['formatted'].tolist(),
  tokenizer=tokenizer,
  max_length=512
)

In [7]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

training_args = TrainingArguments(
  output_dir=str(here('models/gpt2-recipe-finetuned')),
  overwrite_output_dir=True,
  num_train_epochs=4,
)

data_collator = DataCollatorForLanguageModeling(
  tokenizer=tokenizer,
  mlm=False,  # GPT-2 is not a masked language model
)

trainer = Trainer(
  model=model,
  args=training_args,
  train_dataset=dataset,
  data_collator=data_collator,
  tokenizer=tokenizer,
)

print("Trainer initialized")
print(f"Number of trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

C:\Users\Plancha\AppData\Local\Temp\ipykernel_35060\3010357902.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Trainer initialized
Number of trainable parameters: 124,440,576


In [ ]:
from pathlib import Path

model_save_path = here('models/gpt2-recipe-finetuned')
if Path(model_save_path).exists():
  print("Loading existing fine-tuned model...")
  model = AutoModelForCausalLM.from_pretrained(model_save_path)
  tokenizer = AutoTokenizer.from_pretrained(model_save_path)
else:
  # Start fine-tuning
  print("Starting fine-tuning...")
  trainer.train()

  # Save the fine-tuned model
  trainer.save_model(model_save_path)
  tokenizer.save_pretrained(model_save_path)
  print(f"Fine-tuned model saved to: {model_save_path}")

Starting fine-tuning...


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
500,1.926300


Fine-tuned model saved to: c:\Users\Plancha\Desktop\AML-homework\models\gpt2-recipe-finetuned


In [ ]:
# Test the fine-tuned model
eval_data = sql.query(f"""
  select title, directions
  from read_csv(
    '{here('data/recipeNLG/recipeNLG.csv')}', 
    header=True,
    delim=',',
    types = {{
      'title': 'VARCHAR',
      'ingredients': 'VARCHAR[]',
      "directions": 'VARCHAR[]'
    }}
  )
  limit 5
  offset 1001
""")
eval_data = eval_data.df()
eval_data['directions'] = eval_data['directions'].progress_apply(
  lambda x: '\n'.join([f"{i+1} - {step}" for i, step in
  enumerate(x)])
)
eval_data['formatted'] = eval_data.progress_apply(
  lambda row: f"Recipe: {row['title']}\n\nInstructions:\n{row['directions']}\n{tokenizer.eos_token}",
  axis=1
)
eval_dataset = RecipeDataset(
  texts=eval_data['formatted'].tolist(),
  tokenizer=tokenizer,
  max_length=512
)
for i in eval_data['formatted']:
  print(i)
  print("----")
  print("----")

Formatting directions:   0%|          | 0/5 [00:00<?, ?it/s]

Formatting directions:   0%|          | 0/5 [00:00<?, ?it/s]

Recipe: Pepperoni Loaf

Instructions:
1 - Take thawed bread and roll out onto cookie sheet.
2 - Set on door of oven preheated to 350u00b0 and let it rise for 15 to 20 minutes (until puffy).
3 - Lay the slices of Provolone over entire loaf and then layer the pepperoni slices.
4 - Roll bread in a jelly-like roll. Take egg white and baste bread.
5 - Sprinkle sesame seed on top. Bake at 350u00b0 for 30 minutes.
<|endoftext|>
----
----
Recipe: Animal Crackers

Instructions:
1 - Grind oatmeal in blender until fine.
2 - Add honey, salt, flour and soda.
3 - Stir. Cut in butter. Add buttermilk and stir.
4 - Roll very thin.
5 - Cut with animal cookie cutters.
6 - Bake at 400u00b0 until brown (10 to 12 minutes).
<|endoftext|>
----
----
Recipe: Sauerkraut

Instructions:
1 - Cook kraut and water about 10 minutes.
2 - Fry bacon.
3 - Add onion to
4 - drippings and cook until onion is glazed.
5 - Add flour and stir well.
6 - Pour
7 - water from kraut into gravy mixture and add sugar. Add kraut.
8 - Sa

In [ ]:
# Generate text using the fine-tuned model
model.eval()
device = next(model.parameters()).device
inputs = tokenizer(eval_data['formatted'][0].split('\n')[0], return_tensors="pt")
inputs = {k: v.to(device) for k, v in inputs.items()}
with torch.no_grad():
    outputs = model.generate(
        inputs['input_ids'],
        max_new_tokens=100,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
        do_sample=True,
        temperature=0.7,
    )
# Decode and display
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)

The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Recipe: Pepperoni Loaf

Instructions:
1 - Brown onion in oil and garlic.
2 - Add flour, salt, pepper and pepper flakes.
3 - Mix well.
4 - Sprinkle with all ingredients and bake at 350u00b0 for 30 minutes or until a toothpick inserted in center comes out clean.



In [37]:
# compare original model and fine-tuned model
original_model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2")
original_model.eval()
original_model.to(device)
# Generate text using the original model
inputs = tokenizer(eval_data['formatted'][0].split('\n')[0], return_tensors="pt")
inputs = {k: v.to(device) for k, v in inputs.items()}
with torch.no_grad():
  outputs = original_model.generate(
    inputs['input_ids'],
    max_new_tokens=100,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
    do_sample=True,
    temperature=0.7,
  )
# Decode and display
generated_text_original = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text_original)

The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


KeyboardInterrupt: 